# Building the Is Prime? Skill
This notebook demonstrates how to use the Cortex Python SDK to build a simple Skill.  For simple cases, the entire Skill can be defined, tested, and deployed right from a notebook.

In [ ]:
# Common Setup
%run setup.ipynb

## Define the is_prime function


In [ ]:
%%cortex_action --name 'class/is-prime-<your_initials>' --function is_prime

from cortex import Message

def is_prime(params):
    msg = Message(params)
    num = msg.payload.get('num')
    if num == (int(num)):        
        if num > 1:
       # check for factors
           for i in range(2,num):
               if (num % i) == 0:
                   return Message.with_payload({'result': (str(i))+" times " + (str(num/i)) + " is " + (str(num)) + "; Not prime."}).to_params()
                   break
           else:
               return Message.with_payload({'result': (str(num))+ ' is a prime number.'}).to_params()
       
# if input number is less than
# or equal to 1, it is not prime
        else:
           return Message.with_payload({'result': (str(num))+ ' is a prime number.'}).to_params()
    else:
        return Message.with_payload({'result': 'A float is not a prime number'}).to_params()

Build and Deploy Action

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
action = cortex.action('class/is-prime-<your_initials>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _num_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'num': 12345.38}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Simple Calculator" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/is-prime-<your_initials>').title('Is Prime?-<your_initials>').description('Skill which checks if a number is prime by <your_name>')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('num').title('input').parameter(name='num', type='number').all_routing(action, 'result').build()


In the previous step, we referenced an Output called **result**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('result').title('Prime out').parameter(name='isPrime', type='string').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='num', message=Message.with_payload({'num': 50}))
rs.payload